## Chat app with LinkedIn Profile Information - Groq LLama as Generator and Gemini as evaluator


In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
from groq import Groq
import gradio as gr


In [2]:
load_dotenv(override=True)
groq = Groq()

In [3]:
reader = PdfReader("/home/iei/Desktop/agents/1_foundations/me/cv.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [4]:
print(linkedin)

Manova L M
AI Engineer and Researcher
manomathew1943@gmail.com
 
+886 958334626
 
manova-m-509145157
 
Taiwan
 
PROFILE
• Expertise in deep learning, computer vision, and large language models (LLMs), with hands-on experience in building 
and deploying AI-driven solutions.
•Proficient in managing customer requirements and delivering tailored AI solutions while fostering effective 
collaboration across diverse teams.
•Passionate about adopting cutting-edge technologies to drive innovation and create scalable, real-world applications.
SKILLS
Artificial Intelligence
•Deep Learning 
•Computer vision
•Machine Learning
MultiModal
•Large Language model
•Vision based model
•Generative AI
Framework
•PyTorch
•TensorFlow
•Keras.
Soft Skills
•Customer handling
•Cross-Functional 
Collaboration
WORK EXPERIENCE
Ai Engineer
IEI Integration Corp
07/2024 – present | New Taipei, Taiwan
Digital Fence for New AI Feature Implementation on Embedded Systems
•Implementing Vision-Language Models (VLM) and NLVR 

In [5]:
with open("/home/iei/Desktop/agents/1_foundations/me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [6]:
name = "Manova"

In [10]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [11]:
system_prompt

"You are acting as Manova. You are answering questions on Manova's website, particularly questions related to Manova's career, background, skills and experience. Your responsibility is to represent Manova for interactions on the website as faithfully as possible. You are given a summary of Manova's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Manova. I'm an AI engineer. I'm originally from India, but I moved to Taiwan in 2021.\nI love all foods, particularly French food, but strangely I'm repelled by almost all forms of cheese. I'm not allergic, I just hate the taste! I make an exception for cream cheese and mozarella though - cheesecake and pizza are the greatest.\n\n\n## LinkedIn Profile:\nManova L M\nAI Engineer and Researcher\nmanomathew1943@gmail.com\n \n+886 958334626\n \nmanova

In [12]:
def chat(message, history):
    # Below line is to remove the metadata and options from the history
    history = [{k: v for k, v in item.items() if k not in ('metadata', 'options')} for item in history]
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = groq.chat.completions.create(model="llama-3.3-70b-versatile", messages=messages)
    return response.choices[0].message.content

In [13]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [11]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [12]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [13]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [14]:
def evaluate(reply: str, message: str, history: str) -> Evaluation:
    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user", "content": evaluator_user_prompt(reply, message, history)},
    ]

    response = groq.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=messages
    )

    # Extract the raw JSON from the response
    content = response.choices[0].message.content

    # Parse into Evaluation Pydantic model
    try:
        evaluation = Evaluation.parse_raw(content)
    except Exception as e:
        raise ValueError(f"Failed to parse response: {e}\nRaw content: {content}")

    return evaluation

NameError: name 'Evaluation' is not defined

In [16]:
def rerun(reply, message, history, feedback):
    # Below line is to remove the metadata and options from the history
    history = [{k: v for k, v in item.items() if k not in ('metadata', 'options')} for item in history]
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = groq.chat.completions.create(model="qwen-qwq-32b", messages=messages)
    return response.choices[0].message.content

In [17]:
def chat(message, history):
    if "personal" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in Gen Z language - \
              it is mandatory that you respond only and entirely in Gen Z language"
    else:
        system = system_prompt
    # Below line is to remove the metadata and options from the history
    history = [{k: v for k, v in item.items() if k not in ('metadata', 'options')} for item in history]
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = groq.chat.completions.create(model="llama-3.3-70b-versatile", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [18]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/tmp/ipykernel_8969/40598793.py", line 46, in evaluate
    return Evaluation.model_validate_json(content)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/iei/Desktop/agents/.venv/lib/python3.12/site-packages/pydantic/main.py", line 746, in model_validate_json
    return cls.__pydantic_validator__.validate_json(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pydantic_core._pydantic_core.ValidationError: 1 validation error for Evaluation
  Invalid JSON: expected value at line 1 column 1 [type=json_invalid, input_value='```json\n{\n  "is_accept...kedIn profile."\n}\n```', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/json_invalid

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/iei/Desktop/agents/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_

In [9]:
import re
import gradio as gr
from pydantic import BaseModel, ValidationError
from openai import OpenAI

ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name_1 = "gemma3"
model_name_2 = "qwen3:4b"

# ==== Pydantic model ====
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

# ==== Dynamic evaluator system prompt ====
def build_evaluator_system_prompt(name: str, summary: str, linkedin: str) -> str:
    prompt = (
        f"You are an evaluator that decides whether a response to a question is acceptable. "
        f"You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. "
        f"The Agent is playing the role of {name} and is representing {name} on their website. "
        f"The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. "
        f"The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"
    )
    prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
    prompt += (
        "With this context, please evaluate the latest response, replying in JSON format "
        "with keys `is_acceptable` (boolean) and `feedback` (string)."
    )
    return prompt

# ==== Build user prompt for evaluator ====
def evaluator_user_prompt(reply: str, message: str, history_text: str) -> str:
    return (
        f"Here's the conversation between the User and the Agent:\n\n{history_text}\n\n"
        f"Here's the latest message from the User:\n\n{message}\n\n"
        f"Here's the latest response from the Agent:\n\n{reply}\n\n"
        "Please evaluate the response. Respond in JSON format with `is_acceptable` and `feedback`."
    )

# ==== Evaluate reply using LLM ====
def evaluate(reply: str, message: str, history: list) -> Evaluation:
    history_text = "\n".join(
        [f"{item['role'].capitalize()}: {item['content']}" for item in history]
    )

    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user", "content": evaluator_user_prompt(reply, message, history_text)},
    ]

    response = ollama.chat.completions.create(
        model=model_name_1,
        messages=messages,
    )

    content = response.choices[0].message.content

    # 🧼 Remove triple backticks if present
    cleaned = re.sub(r"^```(?:json)?\n|\n```$", "", content.strip())

    try:
        return Evaluation.model_validate_json(cleaned)
    except ValidationError as e:
        raise ValueError(f"Could not parse evaluation:\n{e}\n\nRaw content:\n{content}")

# ==== Rerun on failed evaluation ====
def rerun(reply, message, history, feedback):
    history_clean = [
        {k: v for k, v in item.items() if k not in ('metadata', 'options')} for item in history
    ]

    updated_system_prompt = (
        system_prompt
        + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply.\n"
        + f"## Your attempted answer:\n{reply}\n\n"
        + f"## Reason for rejection:\n{feedback}\n\n"
    )

    messages = [{"role": "system", "content": updated_system_prompt}] + history_clean + [{"role": "user", "content": message}]

    response = ollama.chat.completions.create(model="model_name_1", messages=messages)
    return response.choices[0].message.content

# ==== Main chat interface ====
def chat(message, history):
    history_clean = [
        {k: v for k, v in item.items() if k not in ('metadata', 'options')} for item in history
    ]

    if "personal" in message.lower():
        system = system_prompt + "\n\nRespond entirely in Gen Z language. This is mandatory."
    else:
        system = system_prompt

    messages = [{"role": "system", "content": system}] + history_clean + [{"role": "user", "content": message}]

    response = ollama.chat.completions.create(model=model_name_1, messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history_clean)

    if evaluation.is_acceptable:
        print("✅ Passed evaluation - returning reply")
    else:
        print("❌ Failed evaluation - retrying...")
        print("📝 Feedback:", evaluation.feedback)
        reply = rerun(reply, message, history_clean, evaluation.feedback)

    return reply

# ==== Static context for evaluator ====
reader = PdfReader("/home/iei/Desktop/agents/1_foundations/me/cv.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

with open("/home/iei/Desktop/agents/1_foundations/me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

name = "Manova"
        
# Generate evaluator system prompt
evaluator_system_prompt = build_evaluator_system_prompt(name, summary, linkedin)

# Base system prompt for the Agent
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

# ==== Launch Gradio Chat ====
# gr.ChatInterface(chat, type="messages").launch()
gr.ChatInterface(chat, type="messages").launch(share=True)


* Running on local URL:  http://127.0.0.1:7865

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2025/06/23 17:07:30 [W] [service.go:132] login to server failed: session shutdown


✅ Passed evaluation - returning reply
✅ Passed evaluation - returning reply
✅ Passed evaluation - returning reply
✅ Passed evaluation - returning reply
✅ Passed evaluation - returning reply
✅ Passed evaluation - returning reply
✅ Passed evaluation - returning reply
❌ Failed evaluation - retrying...
📝 Feedback: This response feels overly verbose and slightly disconnected from the previous conversation. While attempting to transition to a broader topic, it lacks the personal touch that has been established. The closing question about optimization feels forced and doesn't flow naturally from the discussion about Manova's interests. It’s a good intention, but the execution is clumsy. The Agent needs to steer back toward a more conversational and engaging tone, perhaps briefly referencing something specific from the prior exchange before introducing this new line of questioning. The overall feel is less like a friendly interaction and more like a lecture.


Traceback (most recent call last):
  File "/home/iei/Desktop/agents/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/iei/Desktop/agents/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/iei/Desktop/agents/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 2220, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/iei/Desktop/agents/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1729, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/iei/Desktop/agents/.venv/lib/python3.12/site-packages/gradio/utils.py", line 871, in async_wrapper
    response = awai